In [79]:
import csv
import random
import os
import shutil
from os.path import join

In [55]:
# Generate fake data.
MAX_SIZE = 10

NUM_SESSIONS = 2
NUM_WORKSHOPS = 40
NUM_USERS = 200

# Create fake workshops.
workshops = [f'workshop_{c}' for c in range(NUM_WORKSHOPS)]
workshops_session_1 = workshops[:20]
workshops_session_2 = workshops[20:]

# Create fake users.
users = [dict(id=f'id_{num}', name=f'user_{num}', email=f'user_{num}@email.com') for num in range(NUM_USERS)]

# create csv rows.
csv_rows = []
for user in users:
    row = list(user.values())

    # Select random option from each session.
    row.extend(random.choices(workshops_session_1, k=2))
    row.extend(random.choices(workshops_session_2, k=2))
    csv_rows.append(row)

In [62]:
email_users = []
session_1_selection = {}
session_2_selection = {}

def add_user(user_data, opt, session_dict):
    success = False
    # Validate that workshop has not been created.
    if opt not in session_dict.keys():
        session_dict[opt] = [user_data]
        success = True

    # Validate that it is not full.    
    elif opt in session_dict.keys() and len(session_dict[opt_1]) < MAX_SIZE:
        session_dict[opt].append(user_data)
        success = True

    return success

def get_selected_session(user_data, opt_1, opt_2, session_dict):
    selected = 'Not selected'
    success = add_user(user_data, opt_1, session_dict)
    if success is True:
        selected = opt_1
    else:
        success = add_user(user_data, opt_2, session_dict)
        if success is True:
            selected = opt_2
    return selected
  
for row in csv_rows:
    user_data = row[0:3]
    opt_1, opt_2 = row[3:5]
    selected_session_1 = get_selected_session(user_data, opt_1, opt_2, session_1_selection)

    opt_1, opt_2 = row[4:6]
    selected_session_2 = get_selected_session(user_data, opt_1, opt_2, session_2_selection)
    
    email_user = user_data.copy()
    
    email_user.extend([selected_session_1, selected_session_2])
    email_users.append(email_user)

In [84]:
def save_to_csv(file_path, data):
    with open(file_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(data)

FOLDER_NAME = 'selection'
WORKSHOPS_FOLDER = 'workshops'

if os.path.isdir(FOLDER_NAME):
    shutil.rmtree(FOLDER_NAME)

# Make global directory and subdirectories.
[os.mkdir(i) for i in [FOLDER_NAME, join(FOLDER_NAME, WORKSHOPS_FOLDER)]]

file_path = join(FOLDER_NAME, "email_users.csv")
save_to_csv(file_path, email_users)

# Save into csv file the list of users per workshop.
all_sessions = [session_1_selection, session_2_selection]
res = [[save_to_csv(join(FOLDER_NAME, WORKSHOPS_FOLDER, f'{k}.csv'), v) for k,v in s.items()] for s in all_sessions]


